In [14]:
# ML 연습
# DT
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

t_df = pd.read_pickle('./dataset/t_df.pkl')

y_df = t_df.survived
X_df = t_df.drop('survived', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, \
                                                    test_size=0.2, \
                                                    random_state=11)
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)

dt_accuracy = accuracy_score(y_test, dt_pred)
print('dt 예측 정확도 :', dt_accuracy)

dt 예측 정확도 : 0.7633587786259542


In [15]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(random_state=0)
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)

lr_accuracy = accuracy_score(y_test, lr_pred)
print('lr 예측 정확도 :', lr_accuracy)

lr 예측 정확도 : 0.8206106870229007


In [16]:
# 평가 사용자 함수
from sklearn.metrics import accuracy_score, precision_score , recall_score , \
confusion_matrix, f1_score
def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print()
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}'.format(accuracy, precision, recall, f1))
    
# get_clf_eval(y_test , lr_pred)
get_clf_eval(y_test , dt_pred)

오차 행렬
[[127  29]
 [ 33  73]]

정확도: 0.7634, 정밀도: 0.7157, 재현율: 0.6887,    F1: 0.7019


In [17]:
from sklearn.model_selection import KFold

# KFold 교차 검증 수행()
# K개의 데이터 폴드 세트를 만들어서 K번 만큼 각 폴드 세트에 
# 학습과 검증 평가를 반복. K가 5이면 5번 평가를 평균한 결과로 예측 성능 평가
def exec_kfold(clf, folds):
    kfold = KFold(n_splits=folds)
    scores = []

    for iter_count, (train_index, test_index) in enumerate(kfold.split(X_df)):
        X_train, X_test = X_df.values[train_index], X_df.values[test_index]
        y_train, y_test = y_df.values[train_index], y_df.values[test_index]

        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print('교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))
#         print('검증 세트 인덱스: {1}'.format(iter_count, test_index))

    mean_score = np.mean(scores)
    print('평균 정확도: {0:.4f}'.format(mean_score))
    
exec_kfold(dt_clf, folds=10)

교차 검증 0 정확도: 0.7634
교차 검증 1 정확도: 0.7634
교차 검증 2 정확도: 0.7481
교차 검증 3 정확도: 0.7710
교차 검증 4 정확도: 0.7099
교차 검증 5 정확도: 0.6870
교차 검증 6 정확도: 0.6947
교차 검증 7 정확도: 0.6870
교차 검증 8 정확도: 0.7405
교차 검증 9 정확도: 0.7692
평균 정확도: 0.7334


In [19]:
# cross_val_scores
# KFold의 일련의 과정을 한꺼번에 수행해주는 API
 
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_df, y_df, cv=20)
for iter_count, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))

print('평균 정확도: {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6364
교차 검증 1 정확도: 0.6667
교차 검증 2 정확도: 0.7576
교차 검증 3 정확도: 0.7727
교차 검증 4 정확도: 0.8636
교차 검증 5 정확도: 0.8182
교차 검증 6 정확도: 0.8182
교차 검증 7 정확도: 0.6515
교차 검증 8 정확도: 0.7424
교차 검증 9 정확도: 0.8462
교차 검증 10 정확도: 0.8462
교차 검증 11 정확도: 0.6462
교차 검증 12 정확도: 0.8462
교차 검증 13 정확도: 0.5077
교차 검증 14 정확도: 0.6615
교차 검증 15 정확도: 0.6154
교차 검증 16 정확도: 0.6000
교차 검증 17 정확도: 0.6769
교차 검증 18 정확도: 0.8308
교차 검증 19 정확도: 0.7231
평균 정확도: 0.7264


In [21]:
# GridSearchCV
# DT 파라미터
# max_depth : 트리의 최대 깊이
# max_features : 최적의 분할을 위해 고려할 최대 피처 개수
# max_leaf_nodes : 말단 노드의 최대 개수
# min_samples_split : 노드를 분할하기 위한 최소한의 샘플 데이터. 디폴트 2. 작게 설정할 수록 분할되는 노드 증가, 과적합 가능성 증가
# min_samples_leaf : 말단 노드가 되기 위한 최소한의 샘플 데이터 수
# 교차 검증을 기반으로 하이퍼 파라미터의 최적 값을 찾게 해줌
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score , \
recall_score ,confusion_matrix, f1_score

parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5],\
              'min_samples_leaf':[1,5,8]}

grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', \
                         cv=5, refit=True)
grid_dclf.fit(X_train, y_train)
print(grid_dclf)
# 교차검증을 기반으로 최적의 하이퍼 파라미터를 찾아줌
print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_)
print('GridSeachCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))

best_dclf = grid_dclf.best_estimator_
print(best_dclf)
dt_pred = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dt_pred)
print('dt 예측 정확도 :', accuracy)

get_clf_eval(y_test , dt_pred)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 5, 8],
                  

### 한글 텍스트 처리 - 감성 분석(네이버 영화평점 150K)

In [22]:
import pandas as pd

news_df = pd.read_csv('./dataset/nsmc/ratings_train.txt', sep='\t')
news_df.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [23]:
news_df.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [24]:
X = news_df.iloc[:,:-1]
y = news_df.iloc[:,-1]
print(X.shape)
y.shape

(150000, 2)


(150000,)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.2,random_state=11)

In [26]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120000 entries, 94561 to 141209
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        120000 non-null  int64 
 1   document  119996 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.7+ MB


In [28]:
# Null, 숫자를 공백으로 처리
import re
X_train = X_train.fillna(' ')
X_train.document = X_train.document.apply(lambda x : re.sub(r"\d+"," ",x))
X_test = X_test.fillna(' ')
X_test.document = X_test.document.apply( lambda x : re.sub(r"\d+", " ", x) )

In [ ]:
print(X_train.info())
X_test.info()

In [30]:
# morphs() 메서드는 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 
# list 객체로 반환
import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Okt
okt = Okt()
def tw_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko


In [31]:
# 사이킷런의 TfidfVectorizor를 이용, TF-IDF 피처 모델을 생성(10분 소요)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# 위에서 만든 tw_tokenizer() 함수를 tokenizer로 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2),\
                             min_df=3, max_df=0.9) # 상위 10% 피처로 추출하지 않음
tfidf_vect.fit(X_train.document)
tfidf_train = tfidf_vect.transform(X_train.document)

In [35]:
# 교차검증 및 하이퍼파라미터 튜닝
# Logistic Regression 을 이용하여 감성 분석 Classification 수행.
# 로지스틱 회귀의 하이퍼 파라미터 C를 설정
# C는 규제 강도를 조절하는 alpha 값의 역수로 작을 수록 규제 강도가 크며
# Penalty는 규제의 유형을 설정하며 l1 규제와 ㅣ2 규제가 있으며 기본은 l2 임
lr_clf = LogisticRegression(random_state=0)
params = {'C': [1,3.5,4.5,5.5,10]}
gcv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy',\
                      verbose=0)
gcv_lr.fit(tfidf_train, y_train)
print(gcv_lr.best_params_, round(gcv_lr.best_score_,4))

{'C': 3.5} 0.8555


In [33]:
# 테스트 세트를 이용 예측시 학습할 때 적용한 TfidfVectorizer를
# 그대로 사용해야 학습시 설정한 피처 개수와 테스트 데이터를 TfidfVectorizer로
# 변경할 피처 개수가 같아짐
from sklearn.metrics import accuracy_score
tfidf_test = tfidf_vect.transform(X_test.document)
best_estimator = gcv_lr.best_estimator_
lr_preds = best_estimator.predict(tfidf_test)
print('Logistic Regression 정확도: ',accuracy_score(y_test,lr_preds))

Logistic Regression 정확도:  0.859


In [36]:
print(y_test.values[:10])
print(lr_preds[:10])

[0 0 1 0 0 0 0 1 1 1]
[0 0 1 0 0 0 0 1 1 1]


#### Q. RF, DT를 이용하여 네이버 영화평점 감성 분석 Calssification을 수행하세요.
- GridSearchCV를 이용 교차검증(cv=3)과 하이퍼파라미터 튜닝 
 - RF params = {'n_estimators':[50,100, 200], 'max_depth':[2,3,5], 'min_samples_leaf':[1,5,8]}
 - DT params = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5],'min_samples_leaf':[1,5,8]}

In [37]:
# rf을 이용하여 감성 분석 Classification 수행
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(random_state=0)
params = {'n_estimators':[50,100, 200], 'max_depth':[2,3,5], 'min_samples_leaf':[1,5,8]}
gcv_rf = GridSearchCV(rf_clf, param_grid = params, cv=3, scoring= 'accuracy',verbose=1)
gcv_rf.fit(tfidf_train, y_train)
print(gcv_rf.best_params_, round(gcv_rf.best_score_,4))

from sklearn.metrics import accuracy_score
tfidf_test = tfidf_vect.transform(X_test.document)
best_estimator = gcv_rf.best_estimator_
rf_preds = best_estimator.predict(tfidf_test)
print('rf 정확도:', accuracy_score(y_test,rf_preds))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  4.1min finished


{'max_depth': 5, 'min_samples_leaf': 5, 'n_estimators': 200} 0.7538
rf 정확도: 0.7545


In [ ]:
# dt를 이용하여 감성 분석 Classification 수행
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=0)
params = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5],\
              'min_samples_leaf':[1,5,8]}
gcv_dt = GridSearchCV(dt_clf,param_grid = params, cv=3, scoring='accuracy',verbose=0)
gcv_dt.fit(tfidf_train, y_train)
print(gcv_dt.best_params_, round(gcv_dt.best_score_,4))

tfidf_test = tfidf_vect.transform(X_test.document)
best_estimator = gcv_dt.best_estimator_
dt_preds = best_estimator.predict(tfidf_test)
print('dt 정확도:', accuracy_score(y_test,dt_preds))

In [ ]:
# skip
# SVM를 이용하여 감성 분석 Classification 수행
# 벡터 공간으로 매핑하는 함수를 커널이라고 함. 
# kernel='rbf'옵션으로 RBF(Radial Basis Function) 함수를 적용

from sklearn import svm 
from sklearn.metrics import accuracy_score
svm_clf = svm.SVC(kernel='rbf', random_state=0)
params = {"C": [1, 10, 100, 1000], "kernel":["rbf"], "gamma":[0.001, 0.0001]}

#[     {"C": [1, 10, 100, 1000], "kernel":["linear"]},   
#     {"C": [1, 10, 100, 1000], "kernel":["sigmoid"], "gamma": [0.001, 0.0001]}]


svm_clf = GridSearchCV( svm_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
svm_clf.fit(tfidf_train, y_train)
print(svm_clf.best_params_, round(svm_clf.best_score_,4))

tfidf_test = tfidf_vect.transform(X_test.document)
svm_best_estimator = svm_clf.best_estimator_
svm_preds = svm_best_estimator(tfidf_test)

print("svm 정확도 : " , accuracy_score(y_test, svm_preds))
# The class sklearn.svm.SVC has parameter max_iter=-1 by default. 
# This causes the optimizer to have no maximum number of iterations

In [ ]:
# skip
# KNN
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

knn_clf = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(knn_clf, tfidf_train, y_train, cv=5)
for iter_count, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))

print('평균 정확도: {0:.4f}'.format(np.mean(scores)))
